In [9]:
import sqlite3
import pandas as pd

In [10]:
fn = 'dump.sqlite'
points = pd.read_sql('select * from points where not banned', sqlite3.connect(fn))


In [11]:
# unnecessary/ unknown features
points = points.drop(columns=['banned','ip'])

In [12]:
points = points[['lat', 'lon', 'wait']]
points = points.dropna()

In [13]:
waiting_time_per_point = points.groupby(["lat", "lon"]).mean()

In [14]:
waiting_time_per_point.to_csv("data/waiting_time_per_point.csv", index=True)